## ReMKiT1D input generator - electron-electron collision operator test for l=0

This example tests relaxation to Maxwellian of the l=0 electron harmonic due to e-e collisions. 

This notebook corresponds to the first test in Section 6.2.2. in the ReMKiT1D code paper. 


In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_ee_coll_test/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 1 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

#### PETSc

Default settings are used here.

#### HDF5

No input HDF5 file is used here and all variables are outputted by default using the wrapper class.

### Normalization setup

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

timeNorm = skn.collTimeei(rk.normalization["eVTemperature"],rk.normalization["density"],rk.normalization["referenceIonZ"])

### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D

uniformVGrid = False

dv0 = 0.0307
cv = 1.03 #Change to 1.025 for the short grid
Nv = 120
vGrid = [dv0/2]
for i in range(1,Nv):
    vGrid.append(vGrid[i-1]*cv)
if uniformVGrid:
    maxV = sum(vGrid)
    vGrid = maxV/Nv*np.ones(Nv)
    
lMax = 0 
gridObj = Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True)

In [ ]:
# Add the grid to the wrapper
rk.grid=gridObj

### Set temperature derivation option and add electron species

In [ ]:
rk.setStandardTextbookOptions([0]) 

rk.addSpecies("e",0)
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0)

### Variable initialization

In [ ]:
T0 = 0.5
n0 = 1.0
T = T0*np.ones(gridObj.numX())
n = n0 * np.ones(gridObj.numX())
W = 3*n*T/2

f = np.zeros([gridObj.numX(),gridObj.numH(),gridObj.numV()])
for i in range(gridObj.numX()):
    f[i,gridObj.getH(0)-1,:] = (T0*np.pi)**(-1.5) * n0 * np.exp(-gridObj.vGrid**2/T0) + (T0*np.pi)**(-1.5) * 0.1 *n0 * np.exp(-(gridObj.vGrid-3)**2/T0) 
    
rk.addVar("f",f,isDistribution=True)
rk.addVar("W",W,units='$10eV$',isDerived=True,derivationRule=sc.derivationRule("energyMoment",["f"]))
rk.addVar("n",n,units='$10^{19} m^{-3}$',isDerived=True,derivationRule=sc.derivationRule("densityMoment",["f"]))
rk.addVar("zeroVar",isDerived=True,outputVar=False)
rk.addVar("T",T,isDerived=True,derivationRule=sc.derivationRule("tempFromEnergye",["W","n","zeroVar"]))
rk.addVar("time",isScalar=True,isDerived=True)

### Adding e-e collision operator model for l = 0

The collision operator is implemented in common_models.py and only used here.

In [ ]:
cm.addEECollIsotropic(modelTag="e-e0",distFunName="f",elTempVar="T",elDensVar="n",wrapper=rk)

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0,convergenceVars=["W","f"],nonlinTol=1e-12) #Everything default except for more lenient absolute convergence tolerance

rk.addIntegrator("BE",integrator)

Set initial timestep length and numbers of allowed implicit and general groups

In [ ]:
initialTimestep =  0.05

rk.setIntegratorGlobalData(1,1,initialTimestep) 

Single integration step


In [ ]:
bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("BE1",bdeStep.dict())

#### Timeloop options

In [ ]:
rk.setFixedNumTimesteps(1200)
rk.setFixedStepOutput(5)

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 240

#### Loading data

Set loadpath to ReMKiT1D directory

In [ ]:
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames,varsToIgnore="zeroVar")
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

### Expected Maxwellian comparison

In [ ]:
f0_analytic=np.zeros((numFiles+1,gridObj.numV()))

for i in range(numFiles+1):
    T = loadedData["T"].values[i,0]
    n = loadedData["n"].values[i,0]
    f0_analytic[i,:] = np.pi**(-1.5) * T ** (-1.5) * n * np.exp(-gridObj.vGrid**2/T) 

In [ ]:
dataName = 'f'

In [ ]:

curveDict = {loadedData.coords["time"].data[t]: hv.Curve(loadedData[dataName][{"time":t,"x":0,"h":0}])*hv.Curve((gridObj.vGrid,f0_analytic[t,:])) for t in range(numFiles+1)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims)

### Paper plots

In [ ]:
def fixed_aspect_ratio_loglin(plot, element):
    '''
    Set a fixed aspect ratio on matplotlib loglog plots 
    regardless of axis units
    '''
    ratio = plot._aspect_param_value
    ax = plot.handles['axis']
    xvals,yvals = ax.axes.get_xlim(),ax.axes.get_ylim()

    xrange = xvals[1]-xvals[0]
    yrange = np.log(yvals[1])-np.log(yvals[0])
    ax.set_aspect(ratio*(xrange/yrange), adjustable='box')

    ax.axes.set_xscale('function', functions=(lambda x : x**2,lambda x : x**(1/2)))
    xticks = np.linspace(xvals[0]**2,xvals[1]**2,10)
    ax.axes.set_xticks(xticks**(1/2),map(str,np.around(xticks,1)))

In [ ]:
plotLin = hv.Overlay([hv.Scatter(loadedData[dataName][{"time":t,"x":0,"h":0}],label='${:.2f}t_0$'.format(loadedData.coords["time"].data[t])).opts(marker=hv.Cycle(['o','x','^']),s=8.0) for t in [0,1,2,20,120,240]])* hv.Curve((gridObj.vGrid,f0_analytic[0,:]),label='Maxwellian').opts(color='k',linestyle='--')
plotLin.opts(xlim=(0,14),logy=True,ylim=(1e-12,2),aspect=30, hooks=[fixed_aspect_ratio_loglin],xlabel="E (eV)",fontsize={'xticks':6.0,'legend':8.0},legend_cols=1,legend_position='top_right')

Rerun with different "cv" values to get the temperature errors in Figure 9b

In [ ]:
tempErrShortGrid=abs(loadedData["T"][:,0]-loadedData["T"][0,0])/loadedData["T"][0,0]
sum(vGrid)

In [ ]:
tempErrLongGrid=abs(loadedData["T"][:,0]-loadedData["T"][0,0])/loadedData["T"][0,0]
sum(vGrid)

In [ ]:
errorPlot = hv.Scatter(tempErrShortGrid,label="$v_{max}=11.27v_{th}$").opts(s=6.0,marker='x')*hv.Scatter(tempErrLongGrid,label="$v_{max}=17.25v_{th}$").opts(s=6.0)
errorPlot.opts(logy=True,ylim=(1e-16,1e-6),aspect=0.5,legend_cols=1,ylabel='$\delta T$')

In [ ]:
hv.output(fig='pdf')
hv.save(plotLin, 'e-eMaxwellian.pdf', dpi=144)
hv.save(errorPlot,'e-eMaxwellianTError.pdf',dpi=144)